###Import libraries, modules

Sequence; core/generic, specific, modules [a-z]

In [ ]:
# Select from extensive list of imports
from __future__ import print_function, division
import argparse
import inspect
import itertools
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os
from PIL import Image
import random
import shutil
from subprocess import check_output
import sys
import time
import unittest

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import torch 
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

# plot inline
%matplotlib inline

# set seeds for reproduction
np.random.seed(0)
torch.manual_seed(0)

# interactive mode on
plt.ion()

###Define data transforms and augmentation before loading

Compose augmentation(random), transforms for training and validation/test sets.
Set size equal to model 
Create tensors

In [ ]:
# Explore dataset
data_dir = '.'  # current dir: 'Deep_Learning_A_Z'
# print(check_output(["ls", data_dir]).decode("utf8"))

# Importing the dataset
if os.path.exists('Churn_Modelling.csv'):
    dataset = pd.read_csv('Churn_Modelling.csv')
 
assert dataset is not None, 'dataframe is loaded'

# Generic function to describe inputs of dataset
def summary_by_type(dataset):
    """Describe stats for each input, grouped by dtype"""
    cls_type = {'cat':['object'], 'num':['int64', 'float64' ]}
    result = {'cat':None, 'num':None}
    for cls in cls_type.keys():
        sumry = [[c, dataset[c].describe()] for c in dataset.columns
                                            if dataset[c].dtype in cls_type[cls]]
        sumry = pd.DataFrame.from_records(sumry)
        data = [sumry[1][:][i] for i, _ in enumerate(sumry[1])]
        result[cls] = pd.DataFrame(data)
    return result

# Sanity check
print(summary_by_type(dataset))



In [ ]:
#

X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

assert isinstance(X, pd.DataFrame), 'dataframe is loaded'
#print(X[:5,],'\n...\n', X[-5:])
print(X.head(15),y.head(15))
print('_'*80)

In [ ]:
#missing data
df_train = X
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# Encoding categorical data

# Make X immutable for hashing (LabelEncoder)
if hasattr(X, 'values'):
    X = X.values  

# instantiate LabelEncoder objects
# fit: set levels and transform: set to ints
le_X_1 = LabelEncoder()                  
X[:, 1] = le_X_1.fit_transform(X[:, 1])  # Geography
le_X_2 = LabelEncoder()
X[:, 2] = le_X_2.fit_transform(X[:, 2])  # Gender

# encode Geography to onehotvector (iso dummyvars?)
# # create dummy vars for Geography to avoid ordinal effect on prediction
onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]  # remove 1 dummy var (dummy var trap) (need n_levels - 1)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Standardize features by removing the mean and scaling to unit variance
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Building and initializing the ANN
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
# Fitting the ANN to the Training set
classifier = build_classifier(optimizer='adam')
classifier.fit(X_train, y_train, batch_size=10, epochs=10)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = y_pred > 0.5
cm = confusion_matrix(y_test, y_pred)
print(cm)

#
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
acc = (tp+tn)/sum(sum(cm))
print(tn, fp, fn, tp)
print(acc)


In [ ]:
# Tuning the ANN
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [8, 16, 32],
              'epochs': [10],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_



In [ ]:
print(best_parameters)
print(best_accuracy)

In [ ]:
# Build best model for prediction
classifier = build_classifier('adam')
classifier.fit(X_train, y_train, batch_size = 8, epochs = 20)

# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [ ]:
print(new_prediction)

### Now with Pytorch

In [ ]:
# https://lirnli.wordpress.com/2017/09/03/one-hot-encoding-in-pytorch/
# def one_hot(batch, depth):
#     ones = torch.sparse.torch.eye(depth)
#     return ones.index_select(0,batch)
# 
# t = torch.LongTensor(X[:, 1])
# one_hot(t, 3)


In [ ]:
# All torchvision pre-trained models expect input images normalized in the same way, 
# i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), 
# where H and W are expected to be at least 224. 
# The images have to be loaded into a range of [0, 1] and then normalized using 
MEAN, SD = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]


# Generic function to build transforms
def transform_composer(img_size=0, val_size=0, **kwargs):
    """Build composed data transforms.
    
    :args: e.g.: img_size=224, val_size=256
           kwargs: boolean list of transforms in correct order:
           resize=True, c_crop=True, r_crop=True, flip=True, rotate=True, 
           tensor=True, normalize=True
    :return: composed transform"""
    train_dict = {
         'r_crop': transforms.RandomResizedCrop(img_size),
         'flip': transforms.RandomHorizontalFlip(),
         'rotate': transforms.RandomRotation(5),
         'tensor': transforms.ToTensor(),      
         'normalize': transforms.Normalize(MEAN, SD)
    }
    val_dict = {
         'resize': transforms.Resize(val_size),
         'c_crop': transforms.CenterCrop(img_size),
         'tensor': transforms.ToTensor(),      
         'normalize': transforms.Normalize(MEAN, SD)
    }
    train_transforms = [train_dict[k] for k, v in kwargs.items() if v and k in train_dict]
    val_transforms = [val_dict[k] for k, v in kwargs.items() if v and k in val_dict]
    
    data_transforms = {
        'train': transforms.Compose(train_transforms),
        'val': transforms.Compose(val_transforms)
        }
    return data_transforms  
    
# Compose transforms
ann_transforms = transform_composer(tensor=True)
# Sanity check: list of transforms
print(repr(ann_transforms['train'].__dict__))
print(repr(ann_transforms['val'].__dict__))


In [ ]:
# Sanity check data directory for subfolders and files
data_dir = 'churn_data'

from subprocess import check_output
print(check_output(["ls", data_dir]).decode("utf8"))

In [ ]:
class MyDataset(data.Dataset):
    """"""

    def __init__(self, inputs, labels):
        """"""
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, index):
        """"""
        inputs = torch.from_numpy(self.inputs[index])
        labels = torch.from_numpy(np.asfarray(self.labels[index]|))
        return inputs, labels
        
    def __len__(self):
        return len(self.inputs.index)





In [ ]:
# Build loaders
M_BATCH = 4
WORKERS = 4
PHASES = ['train', 'val']
TRANSFORMS = ann_transforms

# Split dataset in train and validation sets
inputs_train, inputs_val, labels_train, labels_val =\
            train_test_split(X, y, test_size=0.2, random_state=0)

# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
inputs_train = sc.fit_transform(inputs_train)
inputs_val = sc.transform(inputs_val)

# 
train_set = MyDataset(inputs_train, labels_train)
val_set = MyDataset(inputs_val, labels_val)
# dataloaders = DataLoader(dset, batch_size=M_BATCH, shuffle=True, num_workers=WORKERS)
dset = {'train': train_set, 'val': val_set}
train_set[:5]
# dataloaders


# dataset_sizes = {x: len(dset[x]) for x in ['train', 'val']}
# dataset_sizes
# train = TensorDataset(X_train, y_train)
# val = TensorDataset(X_test, y_test)
# 
# image_datasets = {'train': train, 'val': val}
# 
# dataloaders = {x: data.DataLoader(dset[x], batch_size=M_BATCH,
#                                   shuffle=True, num_workers=WORKERS)
#                                   for x in PHASES}
# dataloaders['train'][0]
# next(iter(dataloaders['train']))



# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           TRANSFORMS[x])
#                                           for x in PHASES}
# 
# dataloaders = {x: data.DataLoader(image_datasets[x], batch_size=M_BATCH,
#                                   shuffle=True, num_workers=WORKERS)
#                                   for x in PHASES}

# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
#class_names = image_datasets['train'].classes

# Sanity check images, class labels
# print((repr(image_datasets['train'].__dict__))[:500])
# print(dataset_sizes)
#print(class_names)

###Visualize data

In [ ]:
# Image viewer function
def imshow(inp, title=None):
    """Show images of Tensors in Dataloader."""
    inp = inp.numpy().transpose((1, 2, 0)) # convert to np
    inp = (SD * inp) + MEAN
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.axis('off')
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

###Build model 
or 
Load pretrained model

In [ ]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.lin1 = nn.Linear(11, 6)
        self.lin2 = nn.Linear(6, 6)
        self.lin3 = nn.Linear(6, 1)
        # self.relu = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()
        
    
    
    def forward(self, inputs):
        out = F.relu(self.lin1(inputs))
        out = F.relu(self.lin2(out))
        out = F.sigmoid(self.lin3(out))
        return out
        

model = ANN()
print(model)
# or  
# ANN_ = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H, D_out),
# )      
        
        

In [ ]:
# Define Network Architecture Class
# IF NOT PRELOADED


###Pretrained model

In [ ]:
# List Pytorch pretrained models
# model_names = sorted(name for name in models.__dict__
#                      if name.islower() 
#                      and not name.startswith("__")
#                      and callable(models.__dict__[name]))
# print(model_names)

In [ ]:
# Generic function to set/define pretrained model
# def pre_model(model, pretrained=True, freeze=True):
#     """"""
#     model = models.__dict__[model](pretrained=pretrained)
#     # freeze parameters in backprop
#     if freeze:
#         for param in model.parameters():
#             param.requires_grad = False
#     num_ftrs = model.fc.in_features    # no. of features in fc layer
#     model.fc = nn.Linear(num_ftrs, 2)  # change out_features to 2 (binary loss)
#     return model
# 
# 
# # Define/set model
# model_all = pre_model('resnet18', pretrained=True, freeze=False)
# model_freeze = pre_model('resnet18', pretrained=True, freeze=True)
# 
# # Sanity check: show model and architecture change
# print(model_all)
# print(models.resnet18(pretrained=True).fc)
# print(model_all.fc)

###Define Loss and optimizer

In [ ]:
# Loss and optimizer functions

# Hyperparameters
LR = 0.001
MOMENTUM = 0.9
DECAY_STEP = 7  # epoch steps between LR decay
DECAY_LR = 0.1

# Loss function for binary classification
criterion = nn.CrossEntropyLoss()

# Optimizer functions
optimizer = optim.Adam(model.parameters(), lr=LR)

# Decay LR
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_LR)

###Define training, validation

In [ ]:
# Check for existing model, load and resume
def load_model(model, optimizer, num_epochs, resume=True):
    """Load and resume from existing model.
    :return: model path"""
    model_name = os.path.join(data_dir,
                              str(model.__class__.__name__)+'_'+
                              str(optimizer.__class__.__name__)+'_'+
                              str(num_epochs)+'.pk1')
    if os.path.exists(model_name) and resume:
        model.load_state_dict(torch.load(model_name))
    return model_name

# Sanity check: path
load_model(model_all, optimizer_all, 20)

In [ ]:
# Generic train helper functions


def b_ward(loss, optimizer, scheduler):
    """Backpropagate loss."""
    optimizer.zero_grad()   # reset gradients
    loss.backward()         # backprop loss
    optimizer.step()        # apply gradients
    

def f_ward(model, phase, criterion, inputs, labels):
    """Forward pass.
    
    http://pytorch.org/docs/master/notes/autograd.html#volatile
    :return: loss and accuracy
    """
    inputs = Variable(inputs, volatile=(phase == 'val'), requires_grad=(phase == 'train'))
    labels = Variable(labels)
    
    # Compute loss and predict label(max log-probability)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    _, preds = torch.max(outputs.data, 1)
    acc = torch.sum(preds == labels.data)
    return loss, acc

    
def train(model, loader, scheduler, criterion, optimizer, phase):
    """Training, validation for each epoch. Forward, backward props and caching metrics.
    
    :return: loss and accuracy"""
    model.train(phase == 'train')
    cache = {'cum_count': 0, 'cum_loss': 0.0, 'cum_acc': 0.0, 
             'avg_loss': 0.0, 'avg_acc': 0.0}

    for i, (inputs, labels) in enumerate(loader):
        
        # forward
        loss, acc = f_ward(model, phase, criterion, inputs, labels)
        
        # backward
        if phase == 'train':
            b_ward(loss, optimizer, scheduler)
            
        # stats
        cache['cum_count'] += inputs.size()[0]
        cache['cum_loss'] += loss.data[0]
        cache['cum_acc'] += acc
        cache['avg_loss'] = cache['cum_loss']/cache['cum_count']
        cache['avg_acc'] = cache['cum_acc']/cache['cum_count']
    return cache['avg_loss'], cache['avg_acc']

In [ ]:
# Generic function for training and evaluation of validation set
def eval_model(model, criterion, optimizer, scheduler, num_epochs=25):
    """Running training and validation."""
    start = time.time()
    
    # Load last best model saved
    model_name = load_model(model, optimizer, num_epochs, resume=True)
    print(model_name)
    best_model = {'model': model_name, 'best_acc': 0.0, 'best_model_wts': model.state_dict()}
    
    for epoch in range(num_epochs):
        #lap = time.time()
        print_header()
    
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            lap = time.time()
            loss, acc = train(model, dataloaders[phase], scheduler, criterion, optimizer, phase)
            
            # update LR decay
            if phase == 'val':
                scheduler.step(loss)
            # update and save best_model
            if phase == 'val' and acc > best_model['best_acc']:
                best_model['best_acc'], best_model['best_model_wts'] = acc, model.state_dict()
                torch.save(model.state_dict(), best_model['model'])
                
            end = time.time()
            print_stat(phase, epoch, loss, acc, end-lap)
            
    finish = time.time()            
    print_model_performance(finish-start, best_model)

    # load best model weights
    model.load_state_dict(best_model['best_model_wts'])
    return model

In [ ]:
# Generic print helper functions


# Helper functions for printing stats
def time_format(secs):
    """Convert seconds to h:mm:ss."""
    m, s = divmod(secs, 60)
    h, m = divmod(m, 60)
    return "%d:%02d:%02d" % (h, m, s)


def print_header():
    """Print header."""
    h_template = """{:8}\t\t {:8}\t\t    {:12}\t {:8}\t\t {:8}"""
    print()
    print(h_template.format('Phase', 'Epoch', 'Loss', 'Accurracy', 'Duration'))
       
       
def print_stat(phase, epoch, loss, acc, duration):
    """Print loss, accuracy and duration at each epoch/phase."""
    p_template = """{:8}\t\t {:8}\t\t {:8.4f}\t\t    {:8.1f}\t\t {:8}"""
    print(p_template.format(phase, epoch, loss, acc*100, time_format(duration)))
    
        
def print_model_performance(duration, best_model):
    """Print best model performance and total duration."""
    print('Training and validation complete in: {:8}\n'
          'Best validation Accuracy: {:2.1f}%\n'
          'Learned model saved: {:16}\n'.format(
           time_format(duration), round(best_model['best_acc']*100), 2), best_model['model'])

###Train model


In [ ]:
EPOCHS = 2

# Train and evaluate validation set
model_all = eval_model(model, criterion, optimizer, 
                       exp_lr_scheduler, num_epochs=EPOCHS)

###Evaluate performance

Tune hypermparameters

In [ ]:
# TODO

###Predict

In [ ]:
# Predict per batch
def pred_batch(model):
    """Predict labels for one batch"""
    inputs, labels = next(iter(dataloaders['val']))
    v_inputs, v_labels = Variable(inputs), Variable(labels)
    
    outputs = model(v_inputs)
    _, preds = torch.max(outputs.data, 1)
    
    return zip(inputs, preds, labels)


# Visualize predictions
def show_pred_batch(model, n_batches, n_columns=M_BATCH):
    """Show from n batches n predictions"""
    for _ in range(n_batches):
        it_batch = list(pred_batch(model))
        title = [(class_names[yhat], (yhat == y)) 
                  for _, yhat, y in it_batch][:n_columns]
        inputs = ([input for input, _, _ in it_batch])[:n_columns]
    
        # Make a grid from batch
        out = torchvision.utils.make_grid(inputs, padding=10)
        imshow(out, title)
    return None


# Show predictions
show_pred_batch(model, 5, 2)

